# Análise exploratória

In [ ]:
import csv
import sys
import pandas
import sqlalchemy
import sqlalchemy.orm as orm
import numpy as np
import os
import json
import re
import html
import unicodedata
import ast
from dotenv import load_dotenv
from datetime import datetime
from sqlalchemy import func, case
import matplotlib.pyplot as plt

## Conectar com o banco de dados

A conexão com o banco de dados, modelos e relacionamentos estão definidos no arquivo `database.py`.

### Importar configuração do banco de dados

In [ ]:
# Importar configuração do banco de dados, modelos e sessão
from database import (
    engine,
    session,
    Base,
    Game,
    Developer,
    Publisher,
    Category,
    Genre,
    Tag,
    Language,
    Screenshot,
    Movie,
    game_tag,
    game_category,
    game_developer,
    game_genre,
    game_language,
    game_publisher,
    test_connection
)

# Testar a conexão com o banco de dados
test_connection()

### Funções utilitárias

In [ ]:
def format_number(number):
    """
    Formata um número para ser exibido com separadores de milhares
    Ex: 1000000 -> 1.000.000
    Ex: 1000000000 -> 1.000.000.000
    Ex: 123456.789 -> 123.456,789
    """
    # Converte para string para tratar parte inteira e decimal
    number_str = str(number)

    if "." in number_str:
        integer_part, decimal_part = number_str.split(".")
    else:
        integer_part = number_str
        decimal_part = None

    # Formata a parte inteira com separadores de milhar
    integer_part = "{:,}".format(int(integer_part)).replace(",", ".")

    # Junta novamente com a parte decimal, se existir
    if decimal_part is not None:
        return f"{integer_part},{decimal_part}"

    return integer_part

## Análise de Jogos

### Grátis vs Pagos

A grande maioria dos jogos na Steam são pagos, mas há uma quantidade significativa de jogos gratuitos.

In [ ]:
# Query
result = session.query(
    func.sum(case((Game.price == 0, 1), else_=0)).label("free_games"),
    func.sum(case((Game.price > 0, 1), else_=0)).label("paid_games"),
).first()

free_games, paid_games = result
total_games = free_games + paid_games

print(f"Jogos gratuitos: {free_games}")
print(f"Jogos pagos: {paid_games}")
print(f"Total de jogos: {total_games}")

# Função para mostrar porcentagem + quantidade
def autopct_format(values):
    def my_autopct(pct):
        total = sum(values)
        count = int(round(pct * total / 100.0))
        return f"{pct:.1f}%\n({format_number(count)} jogos)"

    return my_autopct


# Criar gráfico de pizza
labels = ["Gratuitos", "Pagos"]
sizes = [free_games, paid_games]
colors = ["#66b3ff", "#99ff99"]

plt.figure(figsize=(8, 8))
plt.pie(
    sizes,
    labels=labels,
    colors=colors,
    autopct=autopct_format(sizes),
    startangle=90,
)
plt.title(
    "Distribuição de Jogos: Gratuitos vs Pagos",
    fontsize=16,
    fontweight="bold",
)
plt.axis("equal")
plt.show()

### Preço

A Steam possui uma grande variedade de jogos pagos, com preços que variam bastante. Abaixo está um resumo estatístico dos preços dos jogos pagos, considerando apenas títulos com preço maior que zero.

In [ ]:
session.rollback()

In [ ]:
minimal_price = session.query(func.min(Game.price)).filter(Game.price > 0).scalar()
maximal_price = session.query(func.max(Game.price)).filter(Game.price > 0).scalar()
average_price = session.query(func.avg(Game.price)).filter(Game.price > 0).scalar()
median_price = (
    session.query(
        func.percentile_cont(0.5)
        .within_group(Game.price)
    )
    .filter(Game.price > 0)
    .scalar()
)

print(f"Preço mínimo: {minimal_price:.2f}")
print(f"Preço máximo: {maximal_price:.2f}")
print(f"Preço médio: {average_price:.2f}")
print(f"Preço mediano: {median_price:.2f}")



Podemos inferir com os resultados que há uma distribuição assimétrica à direita. A maioria dos jogos custa perto dos 5 dólares, poucos jogos caros puxam a média para cima.

In [ ]:
percentiles = (
    session.query(
        func.percentile_cont([0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
        .within_group(Game.price)
    )
    .filter(Game.price > 0)
    .scalar()
)

p10, p25, p50, p75, p90, p95, p99 = percentiles

print(f"10%: {p10:.2f}")
print(f"25%: {p25:.2f}")
print(f"50%: {p50:.2f}")
print(f"75%: {p75:.2f}")
print(f"90%: {p90:.2f}")
print(f"95%: {p95:.2f}")
print(f"99%: {p99:.2f}")

labels = ["P10", "P25", "P50", "P75", "P90", "P95", "P99"]
values = [p10, p25, p50, p75, p90, p95, p99]

plt.figure(figsize=(8, 4))
plt.plot(labels, values, marker="o")
plt.title("Distribuição de Preços dos Jogos pagos na Steam")
plt.ylabel("Preço (USD)")
plt.xlabel("Percentil")
plt.grid(True)

plt.show()


In [ ]:
# 1. Carregar dados do banco
df = pandas.read_sql(
    session.query(Game.price)
    .filter(Game.price > 0)
    .statement,
    session.bind,
)

# 2. Definir faixas de preço
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
labels = [
    "0–1", "1–2", "2–3", "3–4",
    "4–5", "5–6", "6–7", "7–8",
    "8–9", "9–10", "10–11", "11–12",
    "12–13", "13–14", "14–15", "15–16",
    "16–17", "17–18", "18–19", "19–20",
    "20–21", "21–22", "22–23", "23–24",
    "24–25"
]

df["price_range"] = pandas.cut(
    df["price"],
    bins=bins,
    labels=labels,
    right=False,
)

# 3. Contar jogos por faixa
counts = df["price_range"].value_counts().sort_index()

# 4. Plotar gráfico
plt.figure(figsize=(10, 5))
counts.plot(kind="bar", edgecolor="black")

plt.title("Distribuição de Preços dos Jogos na Steam (0-25 USD)")
plt.xlabel("Faixa de Preço (USD)")
plt.ylabel("Quantidade de Jogos")

plt.grid(axis="y", alpha=0.3)
plt.tight_layout()

plt.show()

A distribuição de preços dos jogos pagos na Steam apresenta forte concentração nas faixas de menor valor, com um picos evidentes entre US$ 4–5, US$ 9-10, US$ 14-15 e US$ 19-20 seguido por frequências elevadas também nas faixas de US$ 0–1, US$ 1–2 e US$ 2–3. Esse padrão confirma a predominância de títulos de baixo custo na plataforma, indicando que a maior parte da oferta está posicionada em valores acessíveis ao consumidor. A presença de múltiplos picos em torno de preços psicológicos (especialmente valores terminados em .99) sugere que as decisões de precificação seguem estratégias de mercado bem estabelecidas, reforçando a hipótese de que o ecossistema da Steam incentiva preços competitivos como forma de maximizar alcance e visibilidade, especialmente para jogos independentes.

Observa-se ainda uma redução progressiva na quantidade de títulos à medida que o preço aumenta, caracterizando uma distribuição assimétrica à direita com cauda longa. Faixas acima de US$ 10 concentram significativamente menos jogos, indicando que preços mais elevados representam um segmento minoritário da oferta total, possivelmente associado a títulos AAA, edições completas ou pacotes de conteúdo. Esse comportamento reforça a noção da Steam como um mercado de alto volume e baixo ticket médio, no qual a mediana e a concentração em faixas inferiores fornecem uma representação mais fiel do preço típico do que métricas baseadas em valores extremos.

In [ ]:
# 1. Carregar dados do banco (preço e proprietários estimados)
df = pandas.read_sql(
    session.query(
        Game.price,
        Game.estimated_owners_lower,
        Game.estimated_owners_upper
    )
    .filter(Game.price <= 80)
    .filter(Game.price > 0)
    .statement,
    session.bind,
)

# 2. Calcular o número médio de proprietários para cada jogo
df['avg_owners'] = (df['estimated_owners_lower'] + df['estimated_owners_upper']) / 2

# 3. Definir faixas de preço
bins = [0, 5, 10, 20, 30, 50, 60, 70, 80]
labels = ["0-5", "5-10", "10-20", "20-30", "30-50", "50-60", "60-70", "70-80"]

df['price_range'] = pandas.cut(
    df['price'],
    bins=bins,
    labels=labels,
    right=False,
)

# 4. Agrupar por faixa de preço e calcular a média de proprietários
avg_owners_by_range = df.groupby('price_range')['avg_owners'].mean().reset_index()
avg_owners_by_range.columns = ['price_range', 'avg_owners']

# 5. Obter informações para o gráfico
total_games = len(df)
price_range_str = f"0-80"

# 6. Criar o gráfico
fig, ax = plt.subplots(figsize=(12, 7))

# Preparar dados para o gráfico
x_labels = avg_owners_by_range['price_range'].astype(str).tolist()
y_values = avg_owners_by_range['avg_owners'].tolist()

# Criar gradiente de cores (de roxo/azul para verde/amarelo)
colors = plt.cm.viridis(np.linspace(0, 1, len(x_labels)))

# Criar gráfico de barras
bars = ax.bar(x_labels, y_values, color=colors, edgecolor='black', linewidth=1.2)

# 7. Adicionar linha de tendência polinomial
x_numeric = np.arange(len(x_labels))
# Ajustar polinômio de grau 3
z = np.polyfit(x_numeric, y_values, 3)
p = np.poly1d(z)
x_trend = np.linspace(0, len(x_labels) - 1, 100)
y_trend = p(x_trend)

ax.plot(x_trend, y_trend, 'r--', linewidth=2, label='Tendência polinomial', alpha=0.8)

# 8. Configurar o gráfico
ax.set_xlabel('Faixa de Preço (USD)', fontsize=12, fontweight='bold')
ax.set_ylabel('Número Médio de Proprietários', fontsize=12, fontweight='bold')
ax.set_title('Proprietários Médios por Faixa de Preço - Análise Detalhada', fontsize=14, fontweight='bold')

# Formatar eixo Y para mostrar valores em K (milhares)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/1000)}K' if x >= 1000 else f'{int(x)}'))

# Adicionar grid
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Adicionar legenda
ax.legend(loc='upper left', fontsize=10)

# Adicionar informações no canto inferior direito
info_text = f'Total de jogos analisados: {format_number(total_games)}\nIntervalo de preço: {price_range_str}'
ax.text(0.98, 0.02, info_text, transform=ax.transAxes, 
        fontsize=9, verticalalignment='bottom', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

# Mostrar valores exatos
print("\nValores detalhados:")
for _, row in avg_owners_by_range.iterrows():
    print(f"{row['price_range']}: {format_number(int(row['avg_owners']))} proprietários médios")

### Consumidor

### Tags

In [ ]:
# Tags mais comuns

LIMIT = 10

# Query para contar quantos jogos têm cada tag
tag_counts = (
    session.query(
        Tag.name,
        func.count(Game.id).label('game_count')
    )
    .join(game_tag, Tag.id == game_tag.c.tag_id)
    .join(Game, game_tag.c.game_id == Game.id)
    .group_by(Tag.id, Tag.name)
    .order_by(func.count(Game.id).desc())
    .limit(LIMIT)
    .all()
)

# Preparar dados para o gráfico
tags = [tag[0].title() for tag in tag_counts]  # Capitalizar primeira letra
counts = [tag[1] for tag in tag_counts]

# Criar gráfico de barras horizontal
plt.figure(figsize=(12, 8))
colors = plt.cm.viridis(np.linspace(0, 1, len(tags)))
bars = plt.barh(tags, counts, color=colors, edgecolor='black', linewidth=1.2)

# Adicionar valores nas barras
for i, (bar, count) in enumerate(zip(bars, counts)):
    plt.text(
        count + max(counts) * 0.01,  # Posição do texto (um pouco à direita da barra)
        bar.get_y() + bar.get_height() / 2,  # Posição vertical (centro da barra)
        format_number(count),
        va='center',
        fontsize=9,
        fontweight='bold'
    )

plt.xlabel('Número de Jogos', fontsize=12, fontweight='bold')
plt.ylabel('Tags', fontsize=12, fontweight='bold')
plt.title(f"Top {LIMIT} Tags Mais Utilizadas na Steam", fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3, linestyle='--')
plt.gca().invert_yaxis()  # Inverter para mostrar a tag mais comum no topo

# Formatar eixo X para mostrar valores em K (milhares) quando apropriado
ax = plt.gca()
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/1000)}K' if x >= 1000 else f'{int(x)}'))

plt.tight_layout()
plt.show()

# Mostrar valores detalhados
print("\nValores detalhados:")
for tag, count in zip(tags, counts):
    print(f"{tag}: {format_number(count)} jogos")

In [ ]:
# Tags menos comuns

LIMIT = 10

# Query para contar quantos jogos têm cada tag
# Ordenamos de forma crescente para pegar as tags menos comuns
tag_counts = (
    session.query(
        Tag.name,
        func.count(Game.id).label('game_count')
    )
    .join(game_tag, Tag.id == game_tag.c.tag_id)
    .join(Game, game_tag.c.game_id == Game.id)
    .group_by(Tag.id, Tag.name)
    .order_by(func.count(Game.id).asc())  # Ordenar de forma crescente
    .limit(LIMIT)
    .all()
)

# Preparar dados para o gráfico
tags = [tag[0].title() for tag in tag_counts]  # Capitalizar primeira letra
counts = [tag[1] for tag in tag_counts]

# Criar gráfico de barras horizontal
plt.figure(figsize=(12, 8))
colors = plt.cm.coolwarm(np.linspace(0, 1, len(tags)))  # Usar paleta diferente para diferenciar
bars = plt.barh(tags, counts, color=colors, edgecolor='black', linewidth=1.2)

# Adicionar valores nas barras
for i, (bar, count) in enumerate(zip(bars, counts)):
    plt.text(
        count + max(counts) * 0.05 if max(counts) > 0 else count + 0.5,  # Posição do texto
        bar.get_y() + bar.get_height() / 2,  # Posição vertical (centro da barra)
        format_number(count),
        va='center',
        fontsize=9,
        fontweight='bold'
    )

plt.xlabel('Número de Jogos', fontsize=12, fontweight='bold')
plt.ylabel('Tags', fontsize=12, fontweight='bold')
plt.title(f"Top {LIMIT} Tags Menos Utilizadas na Steam", fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3, linestyle='--')
# Não inverter o eixo Y para mostrar a tag menos comum no topo (menor valor)

# Formatar eixo X para mostrar valores em K (milhares) quando apropriado
ax = plt.gca()
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/1000)}K' if x >= 1000 else f'{int(x)}'))

plt.tight_layout()
plt.show()

# Mostrar valores detalhados
print("\nValores detalhados:")
for tag, count in zip(tags, counts):
    print(f"{tag}: {format_number(count)} jogos")

In [ ]:
# Evolução das tags ao longos dos anos.
# Tags mais utilizadas por jogos em cada ano.

# Configuração do intervalo de anos para análise
YEAR_RANGE = (2014, 2024)  # (ano_inicio, ano_fim)

# Query para obter tags, anos e contagem de jogos
tag_year_data = (
    session.query(
        Tag.name,
        func.extract('year', Game.release_date).label('year'),
        func.count(Game.id).label('game_count')
    )
    .join(game_tag, Tag.id == game_tag.c.tag_id)
    .join(Game, game_tag.c.game_id == Game.id)
    .filter(Game.release_date.isnot(None))
    .filter(
        func.extract('year', Game.release_date) >= YEAR_RANGE[0],
        func.extract('year', Game.release_date) <= YEAR_RANGE[1]
    )
    .group_by(Tag.id, Tag.name, func.extract('year', Game.release_date))
    .order_by(func.extract('year', Game.release_date), func.count(Game.id).desc())
    .all()
)

# Converter para DataFrame para facilitar manipulação
df_tag_year = pandas.DataFrame([
    {'tag': tag[0], 'year': int(tag[1]), 'count': tag[2]}
    for tag in tag_year_data
])

# Filtrar DataFrame pelo intervalo de anos configurado
df_tag_year = df_tag_year[
    (df_tag_year['year'] >= YEAR_RANGE[0]) & 
    (df_tag_year['year'] <= YEAR_RANGE[1])
]

# Obter o intervalo de anos (usando os valores configurados ou os disponíveis nos dados)
min_year = max(int(df_tag_year['year'].min()), YEAR_RANGE[0]) if len(df_tag_year) > 0 else YEAR_RANGE[0]
max_year = min(int(df_tag_year['year'].max()), YEAR_RANGE[1]) if len(df_tag_year) > 0 else YEAR_RANGE[1]
years = sorted([y for y in df_tag_year['year'].unique() if YEAR_RANGE[0] <= y <= YEAR_RANGE[1]])

print(f"Análise de tags de {min_year} a {max_year} (configurado: {YEAR_RANGE[0]}-{YEAR_RANGE[1]})")
print(f"Total de anos analisados: {len(years)}")

# Para cada ano, pegar as top N tags
TOP_TAGS_PER_YEAR = 5
TOP_TAGS_TO_TRACK = 10  # Top tags gerais para acompanhar ao longo dos anos

# Primeiro, identificar as top tags gerais (mais usadas no total)
top_tags_overall = (
    df_tag_year.groupby('tag')['count']
    .sum()
    .sort_values(ascending=False)
    .head(TOP_TAGS_TO_TRACK)
    .index.tolist()
)

# Criar DataFrame pivoteado com anos como colunas e tags como linhas
df_pivot = df_tag_year.pivot_table(
    index='tag',
    columns='year',
    values='count',
    fill_value=0
)

# Filtrar apenas as top tags para o gráfico
df_top_tags = df_pivot.loc[top_tags_overall]

# Criar gráfico de linha mostrando evolução das top tags
fig, ax = plt.subplots(figsize=(16, 10))

# Cores para cada tag
colors = plt.cm.tab20(np.linspace(0, 1, len(df_top_tags)))

for idx, (tag, row) in enumerate(df_top_tags.iterrows()):
    years_data = [y for y in years if y in row.index]
    counts_data = [row[y] for y in years_data]
    
    ax.plot(
        years_data,
        counts_data,
        marker='o',
        linewidth=2.5,
        markersize=6,
        label=tag.title(),
        color=colors[idx],
        alpha=0.8
    )

ax.set_xlabel('Ano', fontsize=12, fontweight='bold')
ax.set_ylabel('Número de Jogos', fontsize=12, fontweight='bold')
ax.set_title(f'Evolução das Top {TOP_TAGS_TO_TRACK} Tags Mais Utilizadas na Steam ({min_year}-{max_year})', 
             fontsize=14, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
ax.grid(True, alpha=0.3, linestyle='--')

# Formatar eixo Y
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/1000)}K' if x >= 1000 else f'{int(x)}'))

plt.tight_layout()
plt.show()

# Mostrar top tags por ano
print("\n" + "="*80)
print("Top Tags por Ano:")
print("="*80)

for year in years:
    year_data = df_tag_year[df_tag_year['year'] == year].nlargest(TOP_TAGS_PER_YEAR, 'count')
    if len(year_data) > 0:
        print(f"\n{int(year)}:")
        for _, row in year_data.iterrows():
            print(f"  {row['tag'].title():30s} | {format_number(int(row['count'])):>10s} jogos")


In [ ]:
# Análise de tags emergentes
# Tags que mais cresceram em popularidade entre os anos do intervalo

start_year, end_year = (2020, 2024)

# Query para obter contagem de jogos por tag no ano inicial
tags_start = (
    session.query(
        Tag.name,
        func.count(Game.id).label('game_count')
    )
    .join(game_tag, Tag.id == game_tag.c.tag_id)
    .join(Game, game_tag.c.game_id == Game.id)
    .filter(Game.release_date.isnot(None))
    .filter(
        func.extract('year', Game.release_date) == start_year
    )
    .group_by(Tag.id, Tag.name)
    .all()
)

# Query para obter contagem de jogos por tag no ano final
tags_end = (
    session.query(
        Tag.name,
        func.count(Game.id).label('game_count')
    )
    .join(game_tag, Tag.id == game_tag.c.tag_id)
    .join(Game, game_tag.c.game_id == Game.id)
    .filter(Game.release_date.isnot(None))
    .filter(
        func.extract('year', Game.release_date) == end_year
    )
    .group_by(Tag.id, Tag.name)
    .all()
)

# Converter para dicionários para facilitar a comparação
dict_start = {tag[0]: tag[1] for tag in tags_start}
dict_end = {tag[0]: tag[1] for tag in tags_end}

# Obter todas as tags que aparecem em pelo menos um dos anos
all_tags = set(dict_start.keys()) | set(dict_end.keys())

# Calcular crescimento para cada tag
growth_data = []
for tag in all_tags:
    count_start = dict_start.get(tag, 0)
    count_end = dict_end.get(tag, 0)
    
    # Calcular crescimento absoluto e percentual
    absolute_growth = count_end - count_start
    
    # Calcular taxa de crescimento percentual (evitar divisão por zero)
    if count_start > 0:
        percent_growth = ((count_end - count_start) / count_start) * 100
    elif count_end > 0:
        # Tag que não existia no início (crescimento infinito percentual)
        percent_growth = float('inf')
    else:
        percent_growth = 0
    
    growth_data.append({
        'tag': tag,
        'count_start': count_start,
        'count_end': count_end,
        'absolute_growth': absolute_growth,
        'percent_growth': percent_growth
    })

# Converter para DataFrame
df_growth = pandas.DataFrame(growth_data)

# Filtrar tags que tiveram crescimento positivo
df_growth_positive = df_growth[df_growth['absolute_growth'] > 0].copy()

# Ordenar por crescimento absoluto (maior crescimento primeiro)
df_growth_positive = df_growth_positive.sort_values('absolute_growth', ascending=False)

# Top N tags emergentes
TOP_EMERGING = 15
df_top_emerging = df_growth_positive.head(TOP_EMERGING)

print(f"Análise de Tags Emergentes ({start_year} → {end_year})")
print("="*80)
print(f"\nTotal de tags analisadas: {len(all_tags)}")
print(f"Tags com crescimento positivo: {len(df_growth_positive)}")
print(f"\nTop {TOP_EMERGING} Tags Emergentes (por crescimento absoluto):\n")

for idx, row in df_top_emerging.iterrows():
    growth_str = f"+{format_number(int(row['percent_growth']))}%" if row['percent_growth'] != float('inf') else "Novo"
    print(f"{row['tag'].title():30s} | {format_number(int(row['count_start'])):>8s} → {format_number(int(row['count_end'])):>8s} | "
          f"Crescimento: {format_number(int(row['absolute_growth'])):>8s} ({growth_str})")

# Criar visualização 1: Gráfico de barras mostrando crescimento absoluto
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# Gráfico 1: Crescimento absoluto
tags_names = [tag.title() for tag in df_top_emerging['tag']]
growth_values = df_top_emerging['absolute_growth'].values

colors1 = plt.cm.viridis(np.linspace(0.2, 0.8, len(tags_names)))
bars1 = ax1.barh(tags_names, growth_values, color=colors1, edgecolor='black', linewidth=1.2)

# Adicionar valores nas barras
for bar, value in zip(bars1, growth_values):
    ax1.text(
        value + max(growth_values) * 0.01,
        bar.get_y() + bar.get_height() / 2,
        format_number(int(value)),
        va='center',
        fontsize=9,
        fontweight='bold'
    )

ax1.set_xlabel('Crescimento Absoluto (número de jogos)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Tags', fontsize=12, fontweight='bold')
ax1.set_title(f'Top {TOP_EMERGING} Tags Emergentes - Crescimento Absoluto\n({start_year} → {end_year})', 
              fontsize=13, fontweight='bold')
ax1.grid(axis='x', alpha=0.3, linestyle='--')
ax1.invert_yaxis()

# Formatar eixo X
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/1000)}K' if x >= 1000 else f'{int(x)}'))

# Gráfico 2: Comparação início vs fim
x_pos = np.arange(len(tags_names))
width = 0.35

counts_start = df_top_emerging['count_start'].values
counts_end = df_top_emerging['count_end'].values

bars2_start = ax2.barh(x_pos - width/2, counts_start, width, label=f'{start_year}', 
                       color='#ff9999', edgecolor='black', linewidth=1.2)
bars2_end = ax2.barh(x_pos + width/2, counts_end, width, label=f'{end_year}', 
                     color='#66b3ff', edgecolor='black', linewidth=1.2)

ax2.set_xlabel('Número de Jogos', fontsize=12, fontweight='bold')
ax2.set_ylabel('Tags', fontsize=12, fontweight='bold')
ax2.set_title(f'Comparação: {start_year} vs {end_year}\nTop {TOP_EMERGING} Tags Emergentes', 
              fontsize=13, fontweight='bold')
ax2.set_yticks(x_pos)
ax2.set_yticklabels(tags_names)
ax2.legend(fontsize=10)
ax2.grid(axis='x', alpha=0.3, linestyle='--')
ax2.invert_yaxis()

# Formatar eixo X
ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/1000)}K' if x >= 1000 else f'{int(x)}'))

plt.tight_layout()
plt.show()

# Criar visualização 2: Evolução temporal das top tags emergentes
print("\n" + "="*80)
print("Evolução Temporal das Top Tags Emergentes:")
print("="*80)

# Query para obter dados anuais das top tags emergentes
top_tags_list = df_top_emerging['tag'].tolist()

tag_evolution = (
    session.query(
        Tag.name,
        func.extract('year', Game.release_date).label('year'),
        func.count(Game.id).label('game_count')
    )
    .join(game_tag, Tag.id == game_tag.c.tag_id)
    .join(Game, game_tag.c.game_id == Game.id)
    .filter(Game.release_date.isnot(None))
    .filter(
        func.extract('year', Game.release_date) >= start_year,
        func.extract('year', Game.release_date) <= end_year
    )
    .filter(Tag.name.in_(top_tags_list))
    .group_by(Tag.id, Tag.name, func.extract('year', Game.release_date))
    .order_by(Tag.name, func.extract('year', Game.release_date))
    .all()
)

# Converter para DataFrame
df_evolution = pandas.DataFrame([
    {'tag': tag[0], 'year': int(tag[1]), 'count': tag[2]}
    for tag in tag_evolution
])

# Criar gráfico de evolução temporal
fig, ax = plt.subplots(figsize=(16, 10))

# Obter anos únicos
years_evolution = sorted(df_evolution['year'].unique())

# Cores para cada tag
colors_evo = plt.cm.tab20(np.linspace(0, 1, len(top_tags_list)))

for idx, tag in enumerate(top_tags_list):
    tag_data = df_evolution[df_evolution['tag'] == tag].sort_values('year')
    
    if len(tag_data) > 0:
        ax.plot(
            tag_data['year'].values,
            tag_data['count'].values,
            marker='o',
            linewidth=2.5,
            markersize=7,
            label=tag.title(),
            color=colors_evo[idx],
            alpha=0.8
        )

ax.set_xlabel('Ano', fontsize=12, fontweight='bold')
ax.set_ylabel('Número de Jogos', fontsize=12, fontweight='bold')
ax.set_title(f'Evolução Temporal das Top {TOP_EMERGING} Tags Emergentes ({start_year}-{end_year})', 
             fontsize=14, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9, ncol=1)
ax.grid(True, alpha=0.3, linestyle='--')

# Formatar eixo Y
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/1000)}K' if x >= 1000 else f'{int(x)}'))

plt.tight_layout()
plt.show()

### Gêneros

In [ ]:
# Top 10 gêneros mais populares

LIMIT = 10

# Query para contar quantos jogos têm cada gênero
genre_counts = (
    session.query(
        Genre.name,
        func.count(Game.id).label('game_count')
    )
    .join(game_genre, Genre.id == game_genre.c.genre_id)
    .join(Game, game_genre.c.game_id == Game.id)
    .group_by(Genre.id, Genre.name)
    .order_by(func.count(Game.id).desc())
    .limit(LIMIT)
    .all()
)

# Preparar dados para o gráfico
genres = [genre[0].title() for genre in genre_counts]  # Capitalizar primeira letra
counts = [genre[1] for genre in genre_counts]

# Criar gráfico de barras horizontal
plt.figure(figsize=(12, 8))
colors = plt.cm.viridis(np.linspace(0, 1, len(genres)))
bars = plt.barh(genres, counts, color=colors, edgecolor='black', linewidth=1.2)

# Adicionar valores nas barras
for i, (bar, count) in enumerate(zip(bars, counts)):
    plt.text(
        count + max(counts) * 0.01,  # Posição do texto (um pouco à direita da barra)
        bar.get_y() + bar.get_height() / 2,  # Posição vertical (centro da barra)
        format_number(count),
        va='center',
        fontsize=9,
        fontweight='bold'
    )

plt.xlabel('Número de Jogos', fontsize=12, fontweight='bold')
plt.ylabel('Gêneros', fontsize=12, fontweight='bold')
plt.title(f"Top {LIMIT} Gêneros Mais Populares na Steam", fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3, linestyle='--')
plt.gca().invert_yaxis()  # Inverter para mostrar o gênero mais popular no topo

# Formatar eixo X para mostrar valores em K (milhares) quando apropriado
ax = plt.gca()
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/1000)}K' if x >= 1000 else f'{int(x)}'))

plt.tight_layout()
plt.show()

# Mostrar valores detalhados
print("\nValores detalhados:")
for genre, count in zip(genres, counts):
    print(f"{genre}: {format_number(count)} jogos")


In [ ]:
# Distribuição de generos

TOP_GENRES = 10

# Query para contar quantos jogos têm cada gênero (todos os gêneros)
all_genre_counts = (
    session.query(
        Genre.name,
        func.count(Game.id).label('game_count')
    )
    .join(game_genre, Genre.id == game_genre.c.genre_id)
    .join(Game, game_genre.c.game_id == Game.id)
    .group_by(Genre.id, Genre.name)
    .order_by(func.count(Game.id).desc())
    .all()
)

# Separar top 10 e calcular "Outros"
top_genres = all_genre_counts[:TOP_GENRES]
others_count = sum(genre[1] for genre in all_genre_counts[TOP_GENRES:])

# Preparar dados para o gráfico
genres_labels = [genre[0].title() for genre in top_genres]
genres_counts = [genre[1] for genre in top_genres]

# Adicionar "Outros" se houver
if others_count > 0:
    genres_labels.append("Outros")
    genres_counts.append(others_count)

# Calcular total para porcentagens
total = sum(genres_counts)

# Função para mostrar apenas porcentagem no gráfico
def autopct_format_pct(pct):
    return f"{pct:.1f}%"

# Criar gráfico de pizza
plt.figure(figsize=(10, 10))
colors = plt.cm.Set3(np.linspace(0, 1, len(genres_labels)))

plt.pie(
    genres_counts,
    labels=genres_labels,
    colors=colors,
    autopct=autopct_format_pct,
    startangle=90,
)

plt.title(
    "Distribuição de Jogos por Gênero no Steam (Top 10 + Outros)",
    fontsize=16,
    fontweight="bold",
)
plt.axis("equal")
plt.tight_layout()
plt.show()

# Mostrar valores detalhados
print("\nValores detalhados:")
for genre, count in zip(genres_labels, genres_counts):
    percentage = (count / total) * 100
    print(f"{genre}: {percentage:.1f}% ({format_number(count)} jogos)")

### Categorias

In [ ]:
# Top 10 categorias mais populares

LIMIT = 10

# Query para contar quantos jogos têm cada categoria
category_counts = (
    session.query(
        Category.name,
        func.count(Game.id).label('game_count')
    )
    .join(game_category, Category.id == game_category.c.category_id)
    .join(Game, game_category.c.game_id == Game.id)
    .group_by(Category.id, Category.name)
    .order_by(func.count(Game.id).desc())
    .limit(LIMIT)
    .all()
)

# Preparar dados para o gráfico
categories = [category[0].title() for category in category_counts]  # Capitalizar primeira letra
counts = [category[1] for category in category_counts]

# Criar gráfico de barras horizontal
plt.figure(figsize=(12, 8))
colors = plt.cm.viridis(np.linspace(0, 1, len(categories)))
bars = plt.barh(categories, counts, color=colors, edgecolor='black', linewidth=1.2)

# Adicionar valores nas barras
for i, (bar, count) in enumerate(zip(bars, counts)):
    plt.text(
        count + max(counts) * 0.01,  # Posição do texto (um pouco à direita da barra)
        bar.get_y() + bar.get_height() / 2,  # Posição vertical (centro da barra)
        format_number(count),
        va='center',
        fontsize=9,
        fontweight='bold'
    )

plt.xlabel('Número de Jogos', fontsize=12, fontweight='bold')
plt.ylabel('Categorias', fontsize=12, fontweight='bold')
plt.title(f"Top {LIMIT} Categorias Mais Populares na Steam", fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3, linestyle='--')
plt.gca().invert_yaxis()  # Inverter para mostrar a categoria mais popular no topo

# Formatar eixo X para mostrar valores em K (milhares) quando apropriado
ax = plt.gca()
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/1000)}K' if x >= 1000 else f'{int(x)}'))

plt.tight_layout()
plt.show()

# Mostrar valores detalhados
print("\nValores detalhados:")
for category, count in zip(categories, counts):
    print(f"{category}: {format_number(count)} jogos")

In [ ]:
# Distribuição de categorias

TOP_CATEGORIES = 10

# Query para contar quantos jogos têm cada categoria (todas as categorias)
all_category_counts = (
    session.query(
        Category.name,
        func.count(Game.id).label('game_count')
    )
    .join(game_category, Category.id == game_category.c.category_id)
    .join(Game, game_category.c.game_id == Game.id)
    .group_by(Category.id, Category.name)
    .order_by(func.count(Game.id).desc())
    .all()
)

# Separar top 10 e calcular "Outros"
top_categories = all_category_counts[:TOP_CATEGORIES]
others_count = sum(category[1] for category in all_category_counts[TOP_CATEGORIES:])

# Preparar dados para o gráfico
categories_labels = [category[0].title() for category in top_categories]
categories_counts = [category[1] for category in top_categories]

# Adicionar "Outros" se houver
if others_count > 0:
    categories_labels.append("Outros")
    categories_counts.append(others_count)

# Calcular total para porcentagens
total = sum(categories_counts)

# Função para mostrar apenas porcentagem no gráfico
def autopct_format_pct(pct):
    return f"{pct:.1f}%"

# Criar gráfico de pizza
plt.figure(figsize=(10, 10))
colors = plt.cm.Set3(np.linspace(0, 1, len(categories_labels)))

plt.pie(
    categories_counts,
    labels=categories_labels,
    colors=colors,
    autopct=autopct_format_pct,
    startangle=90,
)

plt.title(
    "Distribuição de Jogos por Categoria no Steam (Top 10 + Outros)",
    fontsize=16,
    fontweight="bold",
)
plt.axis("equal")
plt.tight_layout()
plt.show()

# Mostrar valores detalhados
print("\nValores detalhados:")
for category, count in zip(categories_labels, categories_counts):
    percentage = (count / total) * 100
    print(f"{category}: {percentage:.1f}% ({format_number(count)} jogos)")



### Distribuição de plataformas

In [ ]:
# Gráfico de pizza mostrando a distribuição de plataformas

# Query para contar jogos por plataforma
windows_count = session.query(func.count(Game.id)).filter(Game.windows == True).scalar()
linux_count = session.query(func.count(Game.id)).filter(Game.linux == True).scalar()
mac_count = session.query(func.count(Game.id)).filter(Game.mac == True).scalar()

print(f"Jogos com suporte para Windows: {format_number(windows_count)}")
print(f"Jogos com suporte para Linux: {format_number(linux_count)}")
print(f"Jogos com suporte para Mac: {format_number(mac_count)}")

# Função para mostrar porcentagem + quantidade
def autopct_format(values):
    def my_autopct(pct):
        total = sum(values)
        count = int(round(pct * total / 100.0))
        return f"{pct:.1f}%\n({format_number(count)} jogos)"

    return my_autopct

# Criar gráfico de pizza
labels = ["Windows", "Linux", "Mac"]
sizes = [windows_count, linux_count, mac_count]
colors = ["#0078d4", "#fcc624", "#999999"]  # Cores representativas: Windows azul, Linux amarelo, Mac cinza

plt.figure(figsize=(8, 8))
plt.pie(
    sizes,
    labels=labels,
    colors=colors,
    autopct=autopct_format(sizes),
    startangle=90,
)
plt.title(
    "Distribuição de Jogos por Plataforma",
    fontsize=16,
    fontweight="bold",
)
plt.axis("equal")
plt.show()

## Correlação

In [ ]:
# Análise de correlação entre variáveis numéricas

import seaborn as sns

# Carregar dados numéricos do banco
df_corr = pandas.read_sql(
    session.query(
        Game.price,
        Game.discount,
        Game.dlc_count,
        Game.peak_ccu,
        Game.estimated_owners_lower,
        Game.estimated_owners_upper,
        Game.metacritic_score,
        Game.user_score,
        Game.positive,
        Game.negative,
        Game.achievements,
        Game.recommendations,
        Game.average_playtime_forever,
        Game.average_playtime_2weeks,
        Game.median_playtime_forever,
        Game.median_playtime_2weeks
    ).statement,
    session.bind,
)

# Calcular média de proprietários
df_corr['avg_owners'] = (df_corr['estimated_owners_lower'] + df_corr['estimated_owners_upper']) / 2

# Calcular taxa de aprovação (positive / (positive + negative))
df_corr['approval_rate'] = df_corr['positive'] / (df_corr['positive'] + df_corr['negative'])

# Selecionar apenas variáveis numéricas relevantes para correlação
numeric_cols = [
    'price',
    'discount',
    'dlc_count',
    'peak_ccu',
    'avg_owners',
    'metacritic_score',
    'user_score',
    'positive',
    'negative',
    'achievements',
    'recommendations',
    'average_playtime_forever',
    'average_playtime_2weeks',
    'median_playtime_forever',
    'median_playtime_2weeks',
    'approval_rate'
]

# Filtrar apenas colunas que existem no DataFrame
numeric_cols = [col for col in numeric_cols if col in df_corr.columns]

# Calcular matriz de correlação
correlation_matrix = df_corr[numeric_cols].corr()

# Criar heatmap de correlação
plt.figure(figsize=(16, 12))
sns.heatmap(
    correlation_matrix,
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8},
    vmin=-1,
    vmax=1
)
plt.title('Matriz de Correlação entre Variáveis Numéricas', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Identificar correlações fortes (|r| > 0.5)
print("\n" + "="*80)
print("CORRELAÇÕES FORTES (|r| > 0.5):")
print("="*80)

strong_correlations = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.5 and not pandas.isna(corr_value):
            strong_correlations.append({
                'var1': correlation_matrix.columns[i],
                'var2': correlation_matrix.columns[j],
                'correlation': corr_value
            })

# Ordenar por valor absoluto da correlação
strong_correlations.sort(key=lambda x: abs(x['correlation']), reverse=True)

for corr in strong_correlations:
    print(f"{corr['var1']} ↔ {corr['var2']}: {corr['correlation']:.3f}")

if not strong_correlations:
    print("Nenhuma correlação forte encontrada (|r| > 0.5)")

# Análise de correlações moderadas (0.3 < |r| <= 0.5)
print("\n" + "="*80)
print("CORRELAÇÕES MODERADAS (0.3 < |r| <= 0.5):")
print("="*80)

moderate_correlations = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if 0.3 < abs(corr_value) <= 0.5 and not pandas.isna(corr_value):
            moderate_correlations.append({
                'var1': correlation_matrix.columns[i],
                'var2': correlation_matrix.columns[j],
                'correlation': corr_value
            })

moderate_correlations.sort(key=lambda x: abs(x['correlation']), reverse=True)

for corr in moderate_correlations[:10]:  # Mostrar top 10
    print(f"{corr['var1']} ↔ {corr['var2']}: {corr['correlation']:.3f}")

if not moderate_correlations:
    print("Nenhuma correlação moderada encontrada")

As correlações

In [ ]:
# Visualização de relações específicas com gráficos de dispersão

# Filtrar dados válidos (remover NaN e infinitos)
df_clean = df_corr[numeric_cols].replace([np.inf, -np.inf], np.nan).dropna()

# Criar subplots para visualizar relações interessantes
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Análise de Relações entre Variáveis', fontsize=16, fontweight='bold', y=1.02)

# 1. Preço vs Proprietários médios
ax1 = axes[0, 0]
if 'price' in df_clean.columns and 'avg_owners' in df_clean.columns:
    # Filtrar outliers extremos para melhor visualização
    price_filtered = df_clean[(df_clean['price'] <= 100) & (df_clean['avg_owners'] <= df_clean['avg_owners'].quantile(0.95))]
    ax1.scatter(price_filtered['price'], price_filtered['avg_owners'], alpha=0.3, s=10)
    ax1.set_xlabel('Preço (USD)', fontweight='bold')
    ax1.set_ylabel('Proprietários Médios', fontweight='bold')
    ax1.set_title('Preço vs Proprietários')
    ax1.grid(True, alpha=0.3)
    corr_val = df_clean['price'].corr(df_clean['avg_owners'])
    ax1.text(0.05, 0.95, f'r = {corr_val:.3f}', transform=ax1.transAxes, 
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 2. Peak CCU vs Proprietários médios
ax2 = axes[0, 1]
if 'peak_ccu' in df_clean.columns and 'avg_owners' in df_clean.columns:
    # Filtrar outliers
    ccu_filtered = df_clean[(df_clean['peak_ccu'] <= df_clean['peak_ccu'].quantile(0.95)) & 
                             (df_clean['avg_owners'] <= df_clean['avg_owners'].quantile(0.95))]
    ax2.scatter(ccu_filtered['peak_ccu'], ccu_filtered['avg_owners'], alpha=0.3, s=10, color='green')
    ax2.set_xlabel('Peak CCU', fontweight='bold')
    ax2.set_ylabel('Proprietários Médios', fontweight='bold')
    ax2.set_title('Peak CCU vs Proprietários')
    ax2.grid(True, alpha=0.3)
    corr_val = df_clean['peak_ccu'].corr(df_clean['avg_owners'])
    ax2.text(0.05, 0.95, f'r = {corr_val:.3f}', transform=ax2.transAxes, 
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 3. Positive vs Negative reviews
ax3 = axes[0, 2]
if 'positive' in df_clean.columns and 'negative' in df_clean.columns:
    # Filtrar outliers
    reviews_filtered = df_clean[(df_clean['positive'] <= df_clean['positive'].quantile(0.95)) & 
                                 (df_clean['negative'] <= df_clean['negative'].quantile(0.95))]
    ax3.scatter(reviews_filtered['positive'], reviews_filtered['negative'], alpha=0.3, s=10, color='red')
    ax3.set_xlabel('Avaliações Positivas', fontweight='bold')
    ax3.set_ylabel('Avaliações Negativas', fontweight='bold')
    ax3.set_title('Avaliações Positivas vs Negativas')
    ax3.grid(True, alpha=0.3)
    corr_val = df_clean['positive'].corr(df_clean['negative'])
    ax3.text(0.05, 0.95, f'r = {corr_val:.3f}', transform=ax3.transAxes, 
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 4. Metacritic Score vs User Score
ax4 = axes[1, 0]
if 'metacritic_score' in df_clean.columns and 'user_score' in df_clean.columns:
    ax4.scatter(df_clean['metacritic_score'], df_clean['user_score'], alpha=0.3, s=10, color='purple')
    ax4.set_xlabel('Metacritic Score', fontweight='bold')
    ax4.set_ylabel('User Score', fontweight='bold')
    ax4.set_title('Metacritic vs User Score')
    ax4.grid(True, alpha=0.3)
    corr_val = df_clean['metacritic_score'].corr(df_clean['user_score'])
    ax4.text(0.05, 0.95, f'r = {corr_val:.3f}', transform=ax4.transAxes, 
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 5. Achievements vs Recommendations
ax5 = axes[1, 1]
if 'achievements' in df_clean.columns and 'recommendations' in df_clean.columns:
    # Filtrar outliers
    ach_filtered = df_clean[(df_clean['achievements'] <= df_clean['achievements'].quantile(0.95)) & 
                             (df_clean['recommendations'] <= df_clean['recommendations'].quantile(0.95))]
    ax5.scatter(ach_filtered['achievements'], ach_filtered['recommendations'], alpha=0.3, s=10, color='orange')
    ax5.set_xlabel('Achievements', fontweight='bold')
    ax5.set_ylabel('Recommendations', fontweight='bold')
    ax5.set_title('Achievements vs Recommendations')
    ax5.grid(True, alpha=0.3)
    corr_val = df_clean['achievements'].corr(df_clean['recommendations'])
    ax5.text(0.05, 0.95, f'r = {corr_val:.3f}', transform=ax5.transAxes, 
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 6. Average Playtime Forever vs Median Playtime Forever
ax6 = axes[1, 2]
if 'average_playtime_forever' in df_clean.columns and 'median_playtime_forever' in df_clean.columns:
    # Filtrar outliers
    playtime_filtered = df_clean[(df_clean['average_playtime_forever'] <= df_clean['average_playtime_forever'].quantile(0.95)) & 
                                  (df_clean['median_playtime_forever'] <= df_clean['median_playtime_forever'].quantile(0.95))]
    ax6.scatter(playtime_filtered['average_playtime_forever'], playtime_filtered['median_playtime_forever'], 
                alpha=0.3, s=10, color='teal')
    ax6.set_xlabel('Tempo Médio de Jogo (Forever)', fontweight='bold')
    ax6.set_ylabel('Tempo Mediano de Jogo (Forever)', fontweight='bold')
    ax6.set_title('Tempo Médio vs Mediano de Jogo')
    ax6.grid(True, alpha=0.3)
    corr_val = df_clean['average_playtime_forever'].corr(df_clean['median_playtime_forever'])
    ax6.text(0.05, 0.95, f'r = {corr_val:.3f}', transform=ax6.transAxes, 
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("RESUMO DA ANÁLISE DE CORRELAÇÃO:")
print("="*80)
print(f"\nTotal de variáveis analisadas: {len(numeric_cols)}")
print(f"Total de jogos analisados: {len(df_clean)}")
print(f"\nCorrelações fortes encontradas: {len(strong_correlations)}")
print(f"Correlações moderadas encontradas: {len(moderate_correlations)}")

Os dados do dataset carecem de informações mais detalhadas, o que é normal em datasets formados a partir de webscraping.

In [ ]:
# Análise da quantidade de jogos lançados por ano

# Query para contar jogos por ano
games_by_year = (
    session.query(
        func.extract('year', Game.release_date).label('year'),
        func.count(Game.id).label('game_count')
    )
    .filter(Game.release_date.isnot(None))
    .group_by(func.extract('year', Game.release_date))
    .order_by(func.extract('year', Game.release_date))
    .all()
)

# Preparar dados para o gráfico
years = [int(game[0]) for game in games_by_year]
counts = [game[1] for game in games_by_year]

# Criar gráfico de barras
plt.figure(figsize=(14, 7))
colors = plt.cm.viridis(np.linspace(0, 1, len(years)))
bars = plt.bar(years, counts, color=colors, edgecolor='black', linewidth=1.2)

# Adicionar valores nas barras
for bar, count in zip(bars, counts):
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2.,
        height + max(counts) * 0.01,
        format_number(count),
        ha='center',
        va='bottom',
        fontsize=8,
        fontweight='bold',
        rotation=90
    )

plt.xlabel('Ano', fontsize=12, fontweight='bold')
plt.ylabel('Quantidade de Jogos', fontsize=12, fontweight='bold')
plt.title('Quantidade de Jogos Lançados por Ano na Steam', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3, linestyle='--')

# Rotacionar labels do eixo X para melhor visualização
plt.xticks(years, rotation=45, ha='right')

# Formatar eixo Y para mostrar valores em K (milhares) quando apropriado
ax = plt.gca()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/1000)}K' if x >= 1000 else f'{int(x)}'))

plt.tight_layout()
plt.show()

# Mostrar valores detalhados
print("\n" + "="*80)
print("QUANTIDADE DE JOGOS LANÇADOS POR ANO:")
print("="*80)
total_games = sum(counts)
for year, count in zip(years, counts):
    percentage = (count / total_games) * 100
    print(f"{year}: {format_number(count)} jogos ({percentage:.2f}%)")

print(f"\nTotal de jogos: {format_number(total_games)}")
print(f"Período analisado: {min(years)} - {max(years)}")
print(f"Anos com dados: {len(years)}")

In [ ]:
# gráfico de quantidade de jogos por ano entre 1997 e 207